# 10. Discrete and Mixed Encoding Problems

In this notebook, we demonstrate how to solve problems with non-continuous encodings using VAMOS.
VAMOS supports **Binary**, **Permutation** (Order), **Integer**, and **Mixed** variables.

We will manually configure the **NSGA-II** algorithm with appropriate operators for each encoding.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import optimize, make_problem_selection
from vamos.algorithms import NSGAIIConfig

plt.style.use("ggplot")
print("Discrete encoding problems loaded!")

## 1. Binary Encoding

Binary variables represent yes/no decisions. Common operators:
- **Crossover**: Uniform, one-point, two-point
- **Mutation**: Bit-flip

### 1.1 Binary Knapsack Problem

In [ ]:
# Create binary knapsack problem
knapsack = make_problem_selection("bin_knapsack", n_var=20).instantiate()

print(f"Problem: Binary Knapsack")
print(f"  Variables: {knapsack.n_var} items (binary)")
print(f"  Objectives: f1=Capacity Deviation, f2=Negative Value")
print(f"  Capacity: {knapsack.capacity:.2f}")

In [ ]:
# Configure NSGA-II with Binary Operators
# We use explicit configuration to select the correct operators.
binary_config = (
    NSGAIIConfig.builder()
    .pop_size(50)
    .crossover("uniform", prob=0.9)
    .mutation("bitflip", prob=0.05)
    .selection("tournament", pressure=2)
    
    .build()
)

# Run optimization
print("Running Knapsack Optimization...")
knapsack_result = optimize(
    dict(
        problem=knapsack,
        algorithm="nsgaii",
        algorithm_config=binary_config,
        termination=("n_eval", 3000),
        seed=42,
    )
)

print(f"Found {len(knapsack_result)} Pareto-optimal solutions")

In [ ]:
# Visualize Knapsack Front
F = knapsack_result.F
plt.figure(figsize=(8, 5))
plt.scatter(F[:, 0], -F[:, 1], c="steelblue", s=50, alpha=0.7)
plt.xlabel("Capacity Deviation")
plt.ylabel("Total Value")
plt.title("Binary Knapsack Pareto Front")
plt.grid(True, alpha=0.3)
plt.show()

### 1.2 Binary QUBO Problem

Quadratic Unconstrained Binary Optimization.

In [ ]:
# Create QUBO problem
qubo = make_problem_selection("bin_qubo", n_var=15).instantiate()

print(f"Problem: Binary QUBO ({qubo.n_var} vars)")

# Run optimization (reusing binary_config)
print("Running QUBO Optimization...")
qubo_result = optimize(
    dict(
        problem=qubo,
        algorithm="nsgaii",
        algorithm_config=binary_config,
        termination=("n_eval", 3000),
        seed=42,
    )
)

print(f"Found {len(qubo_result)} solutions")
qubo_result.plot(title="QUBO Front")

## 2. Permutation Encoding (TSP)

Permutation variables represent orderings.
- **Crossover**: Order Crossover (OX)
- **Mutation**: Swap Mutation

In [ ]:
# Create TSP problem
tsp_small = make_problem_selection("tsp6").instantiate()
print(f"Problem: TSP ({tsp_small.n_var} cities)")

# Configure NSGA-II for Permutation
perm_config = (
    NSGAIIConfig.builder().pop_size(50).crossover("ox", prob=0.9).mutation("swap", prob=0.2).selection("tournament").build()
)

# Run optimization
print("Running TSP Optimization...")
tsp_result = optimize(
    dict(
        problem=tsp_small,
        algorithm="nsgaii",
        algorithm_config=perm_config,
        termination=("n_eval", 3000),
        seed=42,
    )
)

print(f"Found {len(tsp_result)} solutions")

In [ ]:
# Visualize TSP Result
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pareto Front
F = tsp_result.F
axes[0].scatter(F[:, 0], F[:, 1], c="forestgreen", s=50, alpha=0.7)
axes[0].set_xlabel("Tour Length")
axes[0].set_ylabel("Max Edge Length")
axes[0].set_title("TSP Pareto Front")

# Best Tour Visualization
best_idx = np.argmin(F[:, 0])
tour = tsp_result.X[best_idx].astype(int)
coords = tsp_small.coordinates

axes[1].scatter(coords[:, 0], coords[:, 1], c="red", s=100, zorder=5)
tour_coords = coords[tour]
tour_coords = np.vstack([tour_coords, tour_coords[0]])
axes[1].plot(tour_coords[:, 0], tour_coords[:, 1], "b-")
axes[1].set_title(f"Best Tour (Len={F[best_idx, 0]:.2f})")

plt.tight_layout()
plt.show()

## 3. Integer Encoding

Integer variables restricted to execution on a grid.
- **Crossover**: SBX (internally handles integers)
- **Mutation**: Polynomial (internally handles integers)

In [ ]:
int_alloc = make_problem_selection("int_alloc", n_var=12).instantiate()
print(f"Problem: Integer Alloc ({int_alloc.n_var} vars)")

# Integer Config
int_config = (
    NSGAIIConfig.builder()
    .pop_size(60)
    .crossover("sbx", prob=0.9, eta=20.0)
    .mutation("pm", prob=1.0 / int_alloc.n_var, eta=20.0)
    
    .build()
)

print("Running Integer Optimization...")
int_res = optimize(
    dict(problem=int_alloc, algorithm="nsgaii", algorithm_config=int_config, termination=("n_eval", 3000), seed=42)
)

int_res.plot(title="Integer Allocation Front")

## 4. Mixed Encoding

Mixed design problems containing real, integer, and categorical variables.

In [ ]:
mixed_design = make_problem_selection("mixed_design", n_var=9).instantiate()
print("Problem: Mixed Design")
print(f"  Vars: Real={mixed_design.n_real}, Int={mixed_design.n_int}, Cat={mixed_design.n_cat}")

# Re-use int_config (SBX/PM work for mixed too, handling types internally)
print("Running Mixed Optimization...")
mixed_res = optimize(
    dict(problem=mixed_design, algorithm="nsgaii", algorithm_config=int_config, termination=("n_eval", 3000), seed=42)
)

mixed_res.plot(title="Mixed Design Front")